# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [47]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'C:/Users/User/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [48]:
df.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [49]:
df['Fare'].head()

0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64

In [50]:
df['Embarked'].head()

0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object

In [51]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Embarked'] = df['Embarked'].fillna('None')
median_df = df.groupby(['Embarked'])['Fare'].median().reset_index()
mode_df = df.groupby(['Embarked'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
min_df=df.groupby(['Embarked'])['Fare'].min().reset_index()
temp = pd.merge(median_df,mode_df,how='left',on='Embarked')
temp = pd.merge(temp,min_df ,on='Embarked')
temp.columns=['Embarked','median_Fare','mode_Fare','min_Fare']
temp

,Embarked,median_Fare,mode_Fare,min_Fare
0,C,29.70,7.2292,4.0125
1,None,80.00,80.0000,80.0000
2,Q,7.75,7.7500,6.7500
3,S,13.00,8.0500,0.0000


In [52]:
df = pd.merge(df, temp, how='left', on=['Embarked'])
df = df.drop(['Embarked'] , axis=1)
df.head(20)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,median_Fare,mode_Fare,min_Fare
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,13.00,8.0500,0.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,29.70,7.2292,4.0125
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,13.00,8.0500,0.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,13.00,8.0500,0.0000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,13.00,8.0500,0.0000
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,7.75,7.7500,6.7500
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,13.00,8.0500,0.0000
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,13.00,8.0500,0.0000
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,13.00,8.0500,0.0000
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,29.70,7.2292,4.0125


In [19]:
min_df

,Embarked,Fare
0,C,4.0125
1,None,80.0000
2,Q,6.7500
3,S,0.0000


In [16]:
median_df

,Embarked,Fare
0,C,29.70
1,None,80.00
2,Q,7.75
3,S,13.00


In [17]:
mode_df

,Embarked,Fare
0,C,7.2292
1,None,80.0000
2,Q,7.7500
3,S,8.0500


In [53]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'median_Fare', 'mode_Fare', 'min_Fare']



,Pclass,Age,SibSp,Parch,Fare,median_Fare,mode_Fare,min_Fare
0,3,22.0,1,0,7.2500,13.0,8.0500,0.0000
1,1,38.0,1,0,71.2833,29.7,7.2292,4.0125
2,3,26.0,0,0,7.9250,13.0,8.0500,0.0000
3,1,35.0,1,0,53.1000,13.0,8.0500,0.0000
4,3,35.0,0,0,8.0500,13.0,8.0500,0.0000


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

以FARE和Embarked來做新特徵無改善

In [54]:
df_orignal = df.drop(['median_Fare', 'mode_Fare', 'min_Fare'] , axis=1)

In [56]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X =MMEncoder.fit_transform(df_orignal)

estimator = LogisticRegression()

cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [57]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X =MMEncoder.fit_transform(df)

estimator = LogisticRegression()

cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7027589317916567